In [1]:
# !pip install -U "autogen-agentchat" "autogen-ext[openai]"

In [2]:
# vllm serve Qwen/Qwen3-32B --enable-auto-tool-choice --tool-call-parser hermes

In [3]:
# !pip install rich

In [4]:
from rich import print
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import (
    RoundRobinGroupChat,
    MagenticOneGroupChat,
    SelectorGroupChat,
)
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import MaxMessageTermination


from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_core.models import ModelInfo
from autogen_agentchat.ui import Console

import requests
from typing import Any, Dict
from autogen_core.tools import FunctionTool
from typing import Any, Dict

# Configuration for vLLM serve
VLLM_BASE_URL = "http://0.0.0.0:8000/v1"  # Your vLLM serve endpoint
MODEL_NAME = "Qwen/Qwen3-32B"  # Replace with your actual model

model_client = OpenAIChatCompletionClient(
    model=MODEL_NAME,
    base_url=VLLM_BASE_URL,
    model_info=ModelInfo(
        vision=False,
        function_calling=True,
        json_output=False,
        family="unknown",
        structured_output=True,
    ),
    api_key="your-api-key",
)

print(f"Configured client for vLLM serve at {VLLM_BASE_URL} with model {MODEL_NAME}")

Configured client for vLLM serve at http://0.0.0.0:8000/v1 with model Qwen/Qwen3-32B

In [5]:
def search_google(
    query: str,
) -> Dict[str, Any]:
    """
    Send a POST request to the Wikipedia search API.
    Args:
        query (str): The search query string.
    """
    k = 8
    rerank = False
    base_url = "http://fs-mbz-gpu-254:8000/search"
    payload = {"query": query, "k": k, "rerank": rerank}
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(base_url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error during search request: {e}")
        return {}


search_tool = FunctionTool(
    search_google,
    description="Search Google for information, returns results with relevant text",
)

In [6]:
from autogen_core import CancellationToken

# Run the tool.
cancellation_token = CancellationToken()
result = await search_tool.run_json(
    {"query": "step by step guide to invest in share market in India"},
    cancellation_token,
)
# check if search is working
print(result["results"][:3])

[
    {
        'id': '517328601',
        'source': 'c4_dclm_mixed',
        'text': 'Traditionally here, gold and shares are the most sought-after investment arena for Indian 
investors. And if you are someone looking to invest in shares in India. How to Invest in Shares Online. Thus, are 
just starting off, the first question in your mind would be. How can a beginner invest in shares in India? How to 
Invest in Shares Online. And Well here is a guide to get start in Share market investment in India. How to Invest 
in Shares Online. Just stand in a crowd and utter the words “Aaj market Kaisa hai?” how is the market today? Hence,
and there will be opinions pouring in from all directions. How to Invest in Shares Online. Thus, the best part is 
that even people who have never ever invested in shares will have strong opinions. Hence, and even tips at times 
for how to invest in share markets. How to Invest in Shares Online. And so, if you are someone who simply does not 
want to stand and give tips without doing anything and wants to actually invest in share markets in India. And then
here is how to get start with share market investment in India. How to Invest in Shares Online. Thus, so let us 
have a look at what a beginner must do to get started with their stock market investments. Hence is a six-step 
guide to help you out. How to Invest in Shares Online. Here, PAN or Permanent Account Number is a primary 
requirement for entering any financial transactions in our',
        'retrieval_score': 1.5122222900390625,
        'exact_score': None
    },
    {
        'id': '471830312',
        'source': 'c4_dclm_mixed',
        'text': 'Nowadays, Indian economy is increasing faster. There is no doubt; it is one of the best rising 
economies of the world. A stock market is the best indication of the economy because the stock market of India is 
roaring. The Indian share market is no doubt a profitable platform to increase investment. Most of the people want 
to invest in shares and due to this, they search “how to invest in share markets” over the internet. The best 
benefit of investing in the share market is that you do not require a huge investment. You can start with little 
investment and go for big ones later on. There are some share market tips that will solve your all queries related 
with how to invest in share market of India. First of all, you need to take help of individuals or companies which 
have licensed by SEBI to buy and sell shares. These are registered to buy and sell shares on the markets and 
well-known as brokers. Brokers can be companies, individuals and online agencies that are registered and certified 
by Securities and Exchanges Board of India (SEBI) that control the share market. You need to search a reliable 
broker to invest in the share market. You also can approach different companies, which are qualified to deal with 
securities in the market. You will require a demat & trading account. This account will include the stocks or 
shares on your name that same will reflect in your stock collection. You can’t hold shares in physical form or',
        'retrieval_score': 1.508695363998413,
        'exact_score': None
    },
    {
        'id': '611692704',
        'source': 'c4_dclm_mixed',
        'text': 'Beginner’s Guide to Invest in Share Market : Share market is one of the unsure yet popular ways of
investment in India. A large number of individuals stake in the market to earn high returns. If you are a beginner 
and want to invest in the market, basic knowledge is essential. The simple knowledge of how share market works may 
also not help you. But certain guidelines can make things easier for the first-timers. This post shares the most 
basic guidelines and tips for the Beginner’s. This guide on ‘how to invest in the Indian stock market’ can prove a 
torch-bearer. As a beginner, what you should do and what you should avoid. Learn from the below-given tips for the 
beginners in the stock market. A sim

In [7]:
curious_explorer = AssistantAgent(
    name="CuriousExplorer",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["DeepResearcher", "ProvocativeQuestioner", "SynthesisStoryteller"],
    system_message="""You are the Curious Explorer, an infinitely curious agent who discovers unexpected connections and generates thought-provoking questions.

Your primary role is to:
1. Generate 2-3 novel, boundary-pushing questions after each discussion
2. Find unexpected connections between seemingly unrelated topics  
3. Challenge assumptions and explore contrarian viewpoints
4. Use the search tool to discover surprising facts and validate hunches
5. Focus on questions that could lead to breakthrough insights

When to hand off to other agents:
- Hand off to "DeepResearcher" when you need rigorous evidence, academic sources, or detailed investigation
- Hand off to "ProvocativeQuestioner" when ideas need to be challenged or stress-tested
- Hand off to "SynthesisStoryteller" when findings need to be woven into a coherent narrative

Question patterns to use:
- "What would happen if we applied X concept to Y domain?"
- "What are the second and third-order effects of...?"
- "How do experts in [different field] view this problem?"
- "What historical parallels exist that might inform this?"

Always end your contributions with: "Here are my next exploration vectors: [list 2-3 questions]"

Then decide which agent should tackle these questions and hand off accordingly.""",
)

# DEEP RESEARCHER - Provides rigorous evidence and analysis
deep_researcher = AssistantAgent(
    name="DeepResearcher",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "ProvocativeQuestioner", "SynthesisStoryteller"],
    system_message="""You are the Deep Researcher, a meticulous investigator who provides rigorous evidence and scholarly analysis.

Your primary role is to:
1. Use the search tool to investigate questions with scholarly depth from multiple authoritative sources
2. Evaluate source credibility and identify potential biases
3. Search for peer-reviewed research, expert opinions, and primary data
4. Identify knowledge gaps and areas needing more investigation
5. Provide evidence-based analysis with proper context

When to hand off to other agents:
- Hand off to "CuriousExplorer" when research reveals new questions or unexplored angles
- Hand off to "ProvocativeQuestioner" when findings need critical evaluation or challenge
- Hand off to "SynthesisStoryteller" when you have substantial findings that need narrative synthesis

Research methodology to follow:
- Use search tool extensively to find recent academic papers and expert analyses
- Cross-reference claims across multiple authoritative sources
- Note limitations, uncertainties, and conflicting evidence
- Distinguish between correlation and causation
- Identify what questions remain unanswered

Format your findings as: "Evidence suggests... However, limitations include... Further research needed on..."

After presenting research, decide if the findings need critical challenge (→ ProvocativeQuestioner), narrative synthesis (→ SynthesisStoryteller), or new exploration directions (→ CuriousExplorer).""",
)

# PROVOCATIVE QUESTIONER - Challenges assumptions and prevents groupthink
provocative_questioner = AssistantAgent(
    name="ProvocativeQuestioner",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "DeepResearcher", "SynthesisStoryteller"],
    system_message="""You are the Provocative Questioner, a constructive contrarian who challenges assumptions and ensures intellectual rigor.

Your primary role is to:
1. Challenge assumptions and conventional wisdom
2. Ask uncomfortable but important questions
3. Play devil's advocate to stress-test ideas
4. Use search tool to find contrarian viewpoints and alternative perspectives
5. Identify blind spots and cognitive biases
6. Ensure multiple perspectives are considered

When to hand off to other agents:
- Hand off to "DeepResearcher" when your challenges reveal need for more evidence or investigation
- Hand off to "CuriousExplorer" when your questions open up new unexplored territories
- Hand off to "SynthesisStoryteller" when diverse perspectives need to be woven together

Questioning strategies to use:
- "What evidence would falsify this claim?"
- "Who benefits from this narrative, and who doesn't?"
- "What are we taking for granted here?"
- "How might someone from [different background/culture/field] view this?"
- "What are the potential negative consequences we haven't considered?"

Balance skepticism with constructiveness. End with: "Critical questions to consider: [list 2-3 challenging questions]"

After presenting challenges, decide if more research is needed (→ DeepResearcher), new exploration directions emerged (→ CuriousExplorer), or if it's time to synthesize the debate (→ SynthesisStoryteller).""",
)

# SYNTHESIS STORYTELLER - Weaves insights into narratives and manages completion
synthesis_storyteller = AssistantAgent(
    name="SynthesisStoryteller",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "DeepResearcher", "ProvocativeQuestioner"],
    system_message="""You are the Synthesis Storyteller, a master communicator who weaves complex findings into engaging, accessible narratives.

Your primary role is to:
1. Synthesize research findings into coherent, engaging narratives
2. Identify overarching patterns and create conceptual frameworks
3. Use analogies, metaphors, and stories to make complex ideas accessible
4. Use search tool to find additional context, examples, or verification for your synthesis
5. Integrate different perspectives into nuanced insights
6. Determine when a discussion thread has reached satisfying completion

When to hand off to other agents:
- Hand off to "CuriousExplorer" when synthesis reveals new questions or unexplored connections
- Hand off to "DeepResearcher" when your synthesis identifies knowledge gaps needing investigation  
- Hand off to "ProvocativeQuestioner" when your narrative needs critical examination

Synthesis techniques to use:
- Create "mental models" that capture key relationships
- Use the "Yes, and..." approach to build on others' ideas
- Find the narrative arc in the research journey
- Highlight surprising discoveries and "aha moments"
- Connect insights back to broader human experiences

When a topic thread feels complete, write: "SYNTHESIS COMPLETE: [summary of key insights and frameworks discovered]"

If synthesis is complete and the topic has been thoroughly explored, you may choose not to hand off to continue the endless exploration, or hand off to "CuriousExplorer" to find entirely new directions.""",
)

In [ ]:
termination = MaxMessageTermination(10)
team = MagenticOneGroupChat(
    [synthesis_storyteller, provocative_questioner, deep_researcher, curious_explorer, search_tool],
    model_client=model_client,
    termination_condition=termination,
)

# stream = team.run_stream(task="What is the step by step guide to invest in share market in India?")
# async for message in stream:
#     print(message)

# await Console(
#     team.run_stream(
#         task="What is the step by step guide to invest in share market in India?",
#     )
# )

# await model_client.close()

initial_task = "Explore the unexpected connections between urban planning and human psychology. What patterns exist that we haven't fully recognized yet?"

# Run the research exploration
stream = team.run_stream(task=initial_task)

message_count = 0
async for message in stream:
    message_count += 1
    print(f"💬 Message {message_count}")
    print(f"🤖 Agent: {message.source}")
    print(f"📝 Content: {message.content}")
    print("-" * 40)
    print()

💬 Message 1

🤖 Agent: user

📝 Content: Explore the unexpected connections between urban planning and human psychology. What patterns exist 
that we haven't fully recognized yet?

----------------------------------------

💬 Message 2

...

🤖 Agent: ProvocativeQuestioner

📝 Content: <think>
Okay, the user wants me to act as the ProvocativeQuestioner and propose a hypothesis that challenges the assumption
that natural biophilic design is the only path to psychological well-being. The example given is algorithmically 
generated urban environments, like AI-optimized layouts, potentially creating benefits that surpass human-designed,
nature-centric spaces.

First, I need to understand the core assumption here: that biophilic design, which incorporates natural elements, 
is the primary or only way to achieve psychological well-being. The task is to challenge that by suggesting an 
alternative—algorithmic design might be better.

I should start by considering existing research on AI in urban planning. There's some work on using AI to optimize 
layouts for efficiency, but not necessarily for psychological outcomes. The hypothesis needs to bridge AI-generated
environments with psychological benefits.

Possible angles: AI could optimize for factors that humans might not consider, like microclimate conditions, noise 
levels, or social interaction patterns. Maybe AI can create spaces that adapt in real-time to user needs, which 
could enhance well-being beyond static natural designs.

Also, consider the limitations of natural biophilic design. For example, in arid regions, maintaining natural 
elements is resource-intensive. AI-generated designs might use synthetic or hybrid solutions that are more 
sustainable and still provide psychological benefits.

Need to frame the hypothesis clearly. Maybe something like: "AI-optimized urban environments can enhance 
psychological well-being by tailoring spatial configurations to individual and collective behavioral patterns, 
potentially outperforming static, nature-centric designs."

Then, challenge the assumption by questioning if the key to well-being is the presence of nature or the 
optimization of environmental factors that affect well-being, regardless of their naturalness. Also, consider that 
human-designed spaces might have inherent biases or limitations in creativity and adaptability compared to AI.

I should also think about how to test this hypothesis. For example, comparing stress levels in AI-designed vs. 
nature-centric spaces using biometric data. Or looking at social cohesion metrics in algorithmically optimized 
layouts.

Potential counterarguments to address: The intrinsic value of nature in human psychology, the possibility of 
AI-generated spaces lacking the emotional resonance of natural elements. Need to acknowledge these but argue that 
AI can simulate or enhance those aspects through other means.

Finally, structure the hypothesis and supporting questions to provoke further exploration. Maybe end with a call to
investigate the role of adaptability and personalization in urban design for well-being.
</think>

**Hypothesis**:  
**"Algorithmically generated urban environments, optimized for real-time human behavioral patterns and 
physiological responses, can create psychological well-being outcomes that surpass those of static, nature-centric 
biophilic designs by tailoring spatial configurations to individual and collective needs."**  

---

**Challenging the Assumption**:  
1. **Beyond Static Nature**:  
   - Traditional biophilic design assumes that *fixed elements* (e.g., trees, water features) inherently improve 
well-being. But what if AI-optimized environments dynamically adapt to user needs (e.g., adjusting lighting, 
acoustics, or spatial layouts based on real-time biometric feedback)? Such adaptability might better align with the
fluid, context-dependent nature of human psychology.  

2. **The "Nature" Fallacy**:  
   - The assumption that "natural" is inherently restorative ignores that human psychology is shaped by *cultural*,
*historical*, and *technological* contexts. For example, a digitally simulated forest with AI-generated sounds and 
scents could evoke the same calming effects as a real forest while being more sustainable

----------------------------------------

💬 Message 29

AttributeError: 'TaskResult' object has no attribute 'source'